In [1]:
import pandas as pd
pd.options.display.max_columns = None

In [2]:
data = pd.read_csv('dataset/frappe/frappe.csv', sep="\t")
appData = pd.read_csv('dataset/frappe/meta.csv', sep="\t")

In [3]:
data.head()

,user,item,cnt,daytime,weekday,isweekend,homework,cost,weather,country,city
0,0,0,1,morning,sunday,weekend,unknown,free,sunny,United States,0
1,1,1,7,afternoon,saturday,weekend,unknown,free,cloudy,Spain,0
2,2,2,6,evening,monday,workday,unknown,free,cloudy,Spain,369
3,3,3,1,sunset,thursday,workday,unknown,free,unknown,United States,1028
4,4,4,428,night,thursday,workday,home,free,sunny,Switzerland,147


In [4]:
data = data.merge(appData, on='item')

### Merge both files

In [5]:
data.head()

,user,item,cnt,daytime,weekday,isweekend,homework,cost,weather,country,city,package,category,downloads,developer,icon,language,description,name,price,rating,short desc
0,0,0,1,morning,sunday,weekend,unknown,free,sunny,United States,0,com.anydo,Productivity,"1,000,000 - 5,000,000",Any.DO,http://d2lh3rxs7crswz.cloudfront.net/com.anydo...,en,"Meet Any.DO, the best way to-do To-do's on And...",Any.DO To-do & Tasks List,Free,4.5,Any.DO helps you remember everything you have ...
1,9,0,1,morning,sunday,weekend,unknown,free,unknown,unknown,0,com.anydo,Productivity,"1,000,000 - 5,000,000",Any.DO,http://d2lh3rxs7crswz.cloudfront.net/com.anydo...,en,"Meet Any.DO, the best way to-do To-do's on And...",Any.DO To-do & Tasks List,Free,4.5,Any.DO helps you remember everything you have ...
2,181,0,26,noon,sunday,weekend,unknown,free,cloudy,Israel,454,com.anydo,Productivity,"1,000,000 - 5,000,000",Any.DO,http://d2lh3rxs7crswz.cloudfront.net/com.anydo...,en,"Meet Any.DO, the best way to-do To-do's on And...",Any.DO To-do & Tasks List,Free,4.5,Any.DO helps you remember everything you have ...
3,248,0,113,noon,wednesday,workday,unknown,free,cloudy,Peru,799,com.anydo,Productivity,"1,000,000 - 5,000,000",Any.DO,http://d2lh3rxs7crswz.cloudfront.net/com.anydo...,en,"Meet Any.DO, the best way to-do To-do's on And...",Any.DO To-do & Tasks List,Free,4.5,Any.DO helps you remember everything you have ...
4,398,0,7,morning,tuesday,workday,unknown,free,cloudy,South Africa,1083,com.anydo,Productivity,"1,000,000 - 5,000,000",Any.DO,http://d2lh3rxs7crswz.cloudfront.net/com.anydo...,en,"Meet Any.DO, the best way to-do To-do's on And...",Any.DO To-do & Tasks List,Free,4.5,Any.DO helps you remember everything you have ...


### check the shape of the DataFrame (rows, columns)

In [6]:
data.shape

(96203, 22)

### Most used apps

In [7]:
data.name.value_counts()[:5]

Facebook              6430
unknown               6153
Gmail                 5840
WhatsApp Messenger    5563
Chrome                5378
Name: name, dtype: int64

In [8]:
feature_cols = ['cnt', 'downloads']

In [9]:
#All rows and the feature columns
X = data.loc[:, feature_cols]

In [10]:
X.shape

(96203, 2)

In [11]:
y = data.rating

In [12]:
y.shape

(96203,)

In [13]:
data.head()

,user,item,cnt,daytime,weekday,isweekend,homework,cost,weather,country,city,package,category,downloads,developer,icon,language,description,name,price,rating,short desc
0,0,0,1,morning,sunday,weekend,unknown,free,sunny,United States,0,com.anydo,Productivity,"1,000,000 - 5,000,000",Any.DO,http://d2lh3rxs7crswz.cloudfront.net/com.anydo...,en,"Meet Any.DO, the best way to-do To-do's on And...",Any.DO To-do & Tasks List,Free,4.5,Any.DO helps you remember everything you have ...
1,9,0,1,morning,sunday,weekend,unknown,free,unknown,unknown,0,com.anydo,Productivity,"1,000,000 - 5,000,000",Any.DO,http://d2lh3rxs7crswz.cloudfront.net/com.anydo...,en,"Meet Any.DO, the best way to-do To-do's on And...",Any.DO To-do & Tasks List,Free,4.5,Any.DO helps you remember everything you have ...
2,181,0,26,noon,sunday,weekend,unknown,free,cloudy,Israel,454,com.anydo,Productivity,"1,000,000 - 5,000,000",Any.DO,http://d2lh3rxs7crswz.cloudfront.net/com.anydo...,en,"Meet Any.DO, the best way to-do To-do's on And...",Any.DO To-do & Tasks List,Free,4.5,Any.DO helps you remember everything you have ...
3,248,0,113,noon,wednesday,workday,unknown,free,cloudy,Peru,799,com.anydo,Productivity,"1,000,000 - 5,000,000",Any.DO,http://d2lh3rxs7crswz.cloudfront.net/com.anydo...,en,"Meet Any.DO, the best way to-do To-do's on And...",Any.DO To-do & Tasks List,Free,4.5,Any.DO helps you remember everything you have ...
4,398,0,7,morning,tuesday,workday,unknown,free,cloudy,South Africa,1083,com.anydo,Productivity,"1,000,000 - 5,000,000",Any.DO,http://d2lh3rxs7crswz.cloudfront.net/com.anydo...,en,"Meet Any.DO, the best way to-do To-do's on And...",Any.DO To-do & Tasks List,Free,4.5,Any.DO helps you remember everything you have ...


### Converts columns that are strings into numbers and store the labels in the labels array
### the index of the labels array corresponds to the column number

In [14]:
labels = [None] * 22
for x in range(0, 22):
    if type(data.iloc[:, x][0]) == str:
        data.iloc[:, x], labels[x] = pd.factorize(data.iloc[:, x])
    else:
        labels[x] = None

In [15]:
data.head()

,user,item,cnt,daytime,weekday,isweekend,homework,cost,weather,country,city,package,category,downloads,developer,icon,language,description,name,price,rating,short desc
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,181,0,26,1,0,0,0,0,2,2,454,0,0,0,0,0,0,0,0,0,0,0
3,248,0,113,1,1,1,0,0,2,3,799,0,0,0,0,0,0,0,0,0,0,0
4,398,0,7,0,2,1,0,0,2,4,1083,0,0,0,0,0,0,0,0,0,0,0
